<a href="https://colab.research.google.com/github/divinenaman/nlp-notebooks/blob/main/grammar-error-correction/grammar_error_correction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install happytransformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 45 kB 3.1 MB/s 
     |████████████████████████████████| 4.7 MB 50.4 MB/s 
     |████████████████████████████████| 1.2 MB 51.4 MB/s 
     |████████████████████████████████| 365 kB 69.6 MB/s 
     |████████████████████████████████| 212 kB 77.4 MB/s 
     |████████████████████████████████| 101 kB 10.9 MB/s 
     |████████████████████████████████| 115 kB 67.7 MB/s 
     |████████████████████████████████| 141 kB 63.5 MB/s 
     |████████████████████████████████| 596 kB 67.3 MB/s 
     |████████████████████████████████| 127 kB 73.7 MB/s 
     |████████████████████████████████| 6.6 MB 59.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:


In [2]:
from happytransformer import HappyTextToText
from datasets import load_dataset
from happytransformer import TTTrainArgs
from google.colab import drive
import csv

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
t5_base = HappyTextToText("T5", "t5-base")

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,
08/03/2022 15:46:02 - INFO - happytransformer.happy_transformer -   Using model: cuda


In [5]:
train_dataset = load_dataset("jfleg", split='validation[:]')

eval_dataset = load_dataset("jfleg", split='test[:]')

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating validation split:   0%|          | 0/755 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/748 [00:00<?, ? examples/s]

Dataset jfleg downloaded and prepared to /root/.cache/huggingface/datasets/jfleg/default/1.0.0/ed4ab2367351fe31949f48849ae6732b164f0d5ea6bb5d4357ff4293ac89511b. Subsequent calls will reuse this data.


In [6]:
for case in train_dataset["corrections"][:5]:
  print(case[0])
  print(case)
  print("--------------------------------------------------------")

So I think we would not be alive if our ancestors did not develop sciences and technologies . 
['So I think we would not be alive if our ancestors did not develop sciences and technologies . ', 'So I think we could not live if older people did not develop science and technologies . ', 'So I think we can not live if old people could not find science and technologies and they did not develop . ', 'So I think we can not live if old people can not find the science and technology that has not been developed . ']
--------------------------------------------------------
Not for use with a car . 
['Not for use with a car . ', 'Do not use in the car . ', 'Car not for use . ', 'Can not use the car . ']
--------------------------------------------------------
Here was no promise of morning , except that we looked up through the trees , and we saw how low the forest had swung . 
['Here was no promise of morning , except that we looked up through the trees , and we saw how low the forest had swung 

In [7]:
csv_path1 = '/content/drive/My Drive/jfleg_train_unprocessed.csv'
csv_path2 = '/content/drive/My Drive/jfleg_train_unprocessed.csv'

def generate_csv(csv_path, dataset):
    with open(csv_path, 'w', newline='') as csvfile:
        writter = csv.writer(csvfile)
        writter.writerow(["input", "target"])
        
        for case in dataset:
     	    # Adding the task's prefix to input   
            input_text = "grammar: " + case["sentence"]
            
            for correction in case["corrections"]:

                # a few of the cases contain blank strings.
                if input_text and correction:
                    writter.writerow([input_text, correction])

generate_csv(csv_path1, train_dataset)
generate_csv(csv_path2, eval_dataset)

In [8]:
# performance of base model
before_result = t5_base.eval(csv_path2)

08/03/2022 15:46:22 - INFO - happytransformer.happy_transformer -   Preprocessing evaluating data...


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-cab50f927744071a/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

***** Running Evaluation *****
  Num examples = 2988
  Batch size = 1


In [9]:
print("Before loss:", before_result.loss)

Before loss: 1.2803919315338135


In [10]:
args = TTTrainArgs(batch_size=8, num_train_epochs = 3)
t5_base.train(csv_path1, args=args)

08/03/2022 15:48:02 - INFO - happytransformer.happy_transformer -   Preprocessing training data...


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-a07a2e0e6767f13a/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

08/03/2022 15:48:06 - INFO - happytransformer.happy_transformer -   Training...
PyTorch: setting up devices
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2988
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1122


Step,Training Loss
500,0.511500
1000,0.363100




Training completed. Do not forget to share your model on huggingface.co/models =)




In [11]:
before_loss = t5_base.eval(csv_path2)

print("After loss: ", before_loss.loss)

08/03/2022 15:52:43 - INFO - happytransformer.happy_transformer -   Preprocessing evaluating data...


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

PyTorch: setting up devices
***** Running Evaluation *****
  Num examples = 2988
  Batch size = 1


After loss:  0.25756698846817017


In [12]:
from happytransformer import TTSettings

beam_settings =  TTSettings(num_beams=5, min_length=1, max_length=20)
example_1 = "grammar: This sentences, has bads grammar and spelling!"
result_1 = t5_base.generate_text(example_1, args=beam_settings)
print(result_1.text)

This sentence has bad grammar and spelling!


In [13]:
example_2 = "grammar: I am enjoys, writtings articles ons AI."

result_2 = t5_base.generate_text(example_2, args=beam_settings)
print(result_2.text)

I enjoy writing articles on AI.


In [14]:
example_2 = "grammar: I was enjoys my trip."

result_2 = t5_base.generate_text(example_2, args=beam_settings)
print(result_2.text)

I enjoyed my trip.


In [15]:
replacements = [
  (" .", "."), 
  (" ,", ","),
  (" '", "'"),
  (" ?", "?"),
  (" !", "!"),
  (" :", "!"),
  (" ;", "!"),
  (" n't", "n't"),
  (" v", "n't"),
  ("2 0 0 6", "2006"),
  ("5 5", "55"),
  ("4 0 0", "400"),
  ("1 7-5 0", "1750"),
  ("2 0 %", "20%"),
  ("5 0", "50"),
  ("1 2", "12"),
  ("1 0", "10"),
  ('" ballast water', '"ballast water')
]

def remove_excess_spaces(text):
  for rep in replacements:
    text = text.replace(rep[0], rep[1])

  return text

In [16]:
csv_path3 = '/content/drive/My Drive/jfleg_train_processed.csv'
csv_path4 = '/content/drive/My Drive/jfleg_train_processed.csv'

def generate_csv(csv_path, dataset):
    with open(csv_path, 'w', newline='') as csvfile:
        writter = csv.writer(csvfile)
        writter.writerow(["input", "target"])
        
        for case in dataset:
     	    # Adding the task's prefix to input   
            input_text = "grammar: " + case["sentence"]
            
            for correction in case["corrections"]:

                # a few of the cases contain blank strings.
                if input_text and correction:
                    input_text = remove_excess_spaces(input_text)
                    correction = remove_excess_spaces(correction)
                    writter.writerow([input_text, correction])

generate_csv(csv_path3, train_dataset)
generate_csv(csv_path4, eval_dataset)

In [17]:
args = TTTrainArgs(batch_size=8, num_train_epochs = 3)
t5_base.train(csv_path3, args=args)

08/03/2022 15:54:28 - INFO - happytransformer.happy_transformer -   Preprocessing training data...


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-05ab2bb4cdcd94f5/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

08/03/2022 15:54:32 - INFO - happytransformer.happy_transformer -   Training...
PyTorch: setting up devices
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2988
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1122


Step,Training Loss
500,0.373100
1000,0.304400




Training completed. Do not forget to share your model on huggingface.co/models =)




In [18]:
before_loss = t5_base.eval(csv_path4)

print("After loss: ", before_loss.loss)

08/03/2022 15:58:58 - INFO - happytransformer.happy_transformer -   Preprocessing evaluating data...


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

PyTorch: setting up devices
***** Running Evaluation *****
  Num examples = 2988
  Batch size = 1


After loss:  0.21924816071987152


In [19]:
example_2 = "grammar: I was enjoys my trip.."

result_2 = t5_base.generate_text(example_2, args=beam_settings)
print(result_2.text)

I enjoyed my trip.
